<a href="https://colab.research.google.com/github/brendontj/tcc-lol-stats/blob/master/LOL_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   

In [2]:
df = pd.read_csv('Result_2.csv')
print(df.shape)

(69860, 60)


#Utils functions



In [3]:
def get_dragon_names(arr):
  dragons = []
  for i in arr:
    if not pd.isna(i):
      for j in i[1:-1].split(','):
        if j not in dragons:
          dragons.append(j)
  return dragons

def number_of_dragons(arr):
  if arr == 0:
    return 0
  return len(arr[1:-1].split(','))

def move_columns(df, cols_to_move, new_index):
    """
    This method re-arranges the columns in a dataframe to place the desired columns at the desired index.
    ex Usage: df = move_columns(df, ['Rev'], 2)   
    :param df:
    :param cols_to_move: The names of the columns to move. They must be a list
    :param new_index: The 0-based location to place the columns.
    :return: Return a dataframe with the columns re-arranged
    """
    other = [c for c in df if c not in cols_to_move]
    start = other[0:new_index]
    end = other[new_index:]
    return df[start + cols_to_move + end]

#Model to predict winner at 05 minutes of game

###Transform dragons data

In [4]:
df_1 = df.copy()

df_1['red_team_dragons'] = df_1['red_team_dragons'].fillna(0)
df_1['blue_team_dragons'] = df_1['blue_team_dragons'].fillna(0)
print(df_1.shape)
df_1['len_red_team_dragons'] = df_1['red_team_dragons'].apply(lambda x: number_of_dragons(x))
df_1['len_blue_team_dragons'] = df_1['blue_team_dragons'].apply(lambda x: number_of_dragons(x))
print(df_1.shape)

(69860, 60)
(69860, 62)


##Filter just MD1, because MD2,MD3 and MD5 are not labeled
### TODO - resolver possivel problema na extracao dos jogos md2/md3/md5

In [5]:
df_f = df_1.loc[df_1['best_of'] == 1]
print(df_f.shape)

(46550, 62)


In [172]:
name_of_columns = [
                                     'game_external_ref',
                                     'patch_version',
                                     'blue_team_ref',
                                     'red_team_ref',
                                     'blue_team_total_gold',
                                     'blue_team_inhibitors',
                                     'blue_team_towers',
                                     'blue_team_barons',
                                     'blue_team_total_kills',	
                                     'red_team_total_gold',
                                     'red_team_inhibitors',	
                                     'red_team_towers',	
                                     'red_team_barons',	
                                     'red_team_total_kills',
                                     'game_timestamp',
                                     'game_number',
                                     'team_a_side',
                                     'team_b_side',
                                     'team_a_record_wins',                                  
                                     'team_b_record_wins',
                                     'team_a_record_losses',
                                     'team_b_record_losses',                                                                     
                                     'best_of',
                                     'red_team_dragons',
                                     'blue_team_dragons',
                                     'participant_id_player_1',
                                     'champion_name_player_1',
                                     'role_player_1',
                                     'level_player_1',
                                     'kills_player_1',
                                     'deaths_player_1',
                                     'assists_player_1',
                                     'total_gold_earned_player_1',
                                     'creep_score_player_1',
                                     'kill_participation_player_1',
                                     'champion_damage_share_player_1',
                                     'wards_placed_player_1',
                                     'wards_destroyed_player_1',
                                     'participant_id_player_2',
                                     'champion_name_player_2',
                                     'role_player_2',
                                     'level_player_2',
                                     'kills_player_2',
                                     'deaths_player_2',
                                     'assists_player_2',
                                     'total_gold_earned_player_2',
                                     'creep_score_player_2',
                                     'kill_participation_player_2',
                                     'champion_damage_share_player_2',
                                     'wards_placed_player_2',
                                     'wards_destroyed_player_2',
                                     'participant_id_player_3',
                                     'champion_name_player_3',
                                     'role_player_3',
                                     'level_player_3',
                                     'kills_player_3',
                                     'deaths_player_3',
                                     'assists_player_3',
                                     'total_gold_earned_player_3',
                                     'creep_score_player_3',
                                     'kill_participation_player_3',
                                     'champion_damage_share_player_3',
                                     'wards_placed_player_3',
                                     'wards_destroyed_player_3',
                                     'participant_id_player_4',
                                     'champion_name_player_4',
                                     'role_player_4',
                                     'level_player_4',
                                     'kills_player_4',
                                     'deaths_player_4',
                                     'assists_player_4',
                                     'total_gold_earned_player_4',
                                     'creep_score_player_4',
                                     'kill_participation_player_4',
                                     'champion_damage_share_player_4',
                                     'wards_placed_player_4',
                                     'wards_destroyed_player_4',
                                     'participant_id_player_5',
                                     'champion_name_player_5',
                                     'role_player_5',
                                     'level_player_5',
                                     'kills_player_5',
                                     'deaths_player_5',
                                     'assists_player_5',
                                     'total_gold_earned_player_5',
                                     'creep_score_player_5',
                                     'kill_participation_player_5',
                                     'champion_damage_share_player_5',
                                     'wards_placed_player_5',
                                     'wards_destroyed_player_5',
                                     'participant_id_player_6',
                                     'champion_name_player_6',
                                     'role_player_6',
                                     'level_player_6',
                                     'kills_player_6',
                                     'deaths_player_6',
                                     'assists_player_6',
                                     'total_gold_earned_player_6',
                                     'creep_score_player_6',
                                     'kill_participation_player_6',
                                     'champion_damage_share_player_6',
                                     'wards_placed_player_6',
                                     'wards_destroyed_player_6',
                                     'participant_id_player_7',
                                     'champion_name_player_7',
                                     'role_player_7',
                                     'level_player_7',
                                     'kills_player_7',
                                     'deaths_player_7',
                                     'assists_player_7',
                                     'total_gold_earned_player_7',
                                     'creep_score_player_7',
                                     'kill_participation_player_7',
                                     'champion_damage_share_player_7',
                                     'wards_placed_player_7',
                                     'wards_destroyed_player_7',                                
                                     'participant_id_player_8',
                                     'champion_name_player_8',
                                     'role_player_8',
                                     'level_player_8',
                                     'kills_player_8',
                                     'deaths_player_8',
                                     'assists_player_8',
                                     'total_gold_earned_player_8',
                                     'creep_score_player_8',
                                     'kill_participation_player_8',
                                     'champion_damage_share_player_8',
                                     'wards_placed_player_8',
                                     'wards_destroyed_player_8',                                     
                                     'participant_id_player_9',
                                     'champion_name_player_9',
                                     'role_player_9',
                                     'level_player_9',
                                     'kills_player_9',
                                     'deaths_player_9',
                                     'assists_player_9',
                                     'total_gold_earned_player_9',
                                     'creep_score_player_9',
                                     'kill_participation_player_9',
                                     'champion_damage_share_player_9',
                                     'wards_placed_player_9',
                                     'wards_destroyed_player_9',                                
                                     'participant_id_player_10',
                                     'champion_name_player_10',
                                     'role_player_10',
                                     'level_player_10',
                                     'kills_player_10',
                                     'deaths_player_10',
                                     'assists_player_10',
                                     'total_gold_earned_player_10',
                                     'creep_score_player_10',
                                     'kill_participation_player_10',
                                     'champion_damage_share_player_10',
                                     'wards_placed_player_10',
                                     'wards_destroyed_player_10',
                                     'game_winner' # Game winner; 0 if team_a win the game and 1 if team_b win the game
                                     ]
df_5_minutes = pd.DataFrame(columns=name_of_columns)

In [173]:
games_ref_list = df_f['game_external_ref'].unique()

for i in range(len(games_ref_list)):
    sub_df = df_f.loc[df_f['game_external_ref'] == games_ref_list[i]]
    
    game_timestamp_list = sub_df['game_timestamp'].unique()
    
    micro_df = sub_df.loc[sub_df['game_timestamp'] == game_timestamp_list[0]]
    
    if int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 1 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 0:
      game_winner = 0
    elif int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 0 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 1:
      game_winner = 1
    
    player_1 = micro_df.iloc[[0]]
    player_2 = micro_df.iloc[[1]]
    player_3 = micro_df.iloc[[2]]
    player_4 = micro_df.iloc[[3]]
    player_5 = micro_df.iloc[[4]]
    player_6 = micro_df.iloc[[5]]
    player_7 = micro_df.iloc[[6]]
    player_8 = micro_df.iloc[[7]]
    player_9 = micro_df.iloc[[8]]
    player_10 = micro_df.iloc[[9]]

    data = [(player_1.game_external_ref).values[0],
            (player_1.patch_version).values[0],
            (player_1.blue_team_ref).values[0],
            (player_1.red_team_ref).values[0],        
            (player_1.blue_team_total_gold).values[0],
            (player_1.blue_team_inhibitors).values[0],
            (player_1.blue_team_towers).values[0],
            (player_1.blue_team_barons).values[0],
            (player_1.blue_team_total_kills).values[0],
            (player_1.red_team_total_gold).values[0],
            (player_1.red_team_inhibitors).values[0],
            (player_1.red_team_towers).values[0],          
            (player_1.red_team_barons).values[0],
            (player_1.red_team_total_kills).values[0],
            (player_1.game_timestamp).values[0],
            (player_1.game_number).values[0],
            (player_1.team_a_side).values[0],
            (player_1.team_b_side).values[0],
            (player_1.team_a_record_wins).values[0],            
            (player_1.team_b_record_wins).values[0],
            (player_1.team_a_record_losses).values[0],
            (player_1.team_b_record_losses).values[0],
            (player_1.best_of).values[0],
            (player_1.len_red_team_dragons).values[0],
            (player_1.len_blue_team_dragons).values[0],
            (player_1.participant_id).values[0],
            (player_1.champion_name).values[0],
            (player_1.role).values[0],
            (player_1.level).values[0],
            (player_1.kills).values[0],
            (player_1.deaths).values[0],
            (player_1.assists).values[0],
            (player_1.total_gold_earned).values[0],
            (player_1.creep_score).values[0],
            (player_1.kill_participation).values[0],
            (player_1.champion_damage_share).values[0],
            (player_1.wards_placed).values[0],
            (player_1.wards_destroyed).values[0],
            (player_2.participant_id).values[0],
            (player_2.champion_name).values[0],
            (player_2.role).values[0],
            (player_2.level).values[0],
            (player_2.kills).values[0],
            (player_2.deaths).values[0],
            (player_2.assists).values[0],
            (player_2.total_gold_earned).values[0],
            (player_2.creep_score).values[0],
            (player_2.kill_participation).values[0],
            (player_2.champion_damage_share).values[0],
            (player_2.wards_placed).values[0],
            (player_2.wards_destroyed).values[0],           
            (player_3.participant_id).values[0],
            (player_3.champion_name).values[0],
            (player_3.role).values[0],
            (player_3.level).values[0],
            (player_3.kills).values[0],
            (player_3.deaths).values[0],
            (player_3.assists).values[0],
            (player_3.total_gold_earned).values[0],
            (player_3.creep_score).values[0],
            (player_3.kill_participation).values[0],
            (player_3.champion_damage_share).values[0],
            (player_3.wards_placed).values[0],
            (player_3.wards_destroyed).values[0],           
            (player_4.participant_id).values[0],
            (player_4.champion_name).values[0],
            (player_4.role).values[0],
            (player_4.level).values[0],
            (player_4.kills).values[0],
            (player_4.deaths).values[0],
            (player_4.assists).values[0],
            (player_4.total_gold_earned).values[0],
            (player_4.creep_score).values[0],
            (player_4.kill_participation).values[0],
            (player_4.champion_damage_share).values[0],
            (player_4.wards_placed).values[0],
            (player_4.wards_destroyed).values[0],           
            (player_5.participant_id).values[0],
            (player_5.champion_name).values[0],
            (player_5.role).values[0],
            (player_5.level).values[0],
            (player_5.kills).values[0],
            (player_5.deaths).values[0],
            (player_5.assists).values[0],
            (player_5.total_gold_earned).values[0],
            (player_5.creep_score).values[0],
            (player_5.kill_participation).values[0],
            (player_5.champion_damage_share).values[0],
            (player_5.wards_placed).values[0],
            (player_5.wards_destroyed).values[0],          
            (player_6.participant_id).values[0],
            (player_6.champion_name).values[0],
            (player_6.role).values[0],
            (player_6.level).values[0],
            (player_6.kills).values[0],
            (player_6.deaths).values[0],
            (player_6.assists).values[0],
            (player_6.total_gold_earned).values[0],
            (player_6.creep_score).values[0],
            (player_6.kill_participation).values[0],
            (player_6.champion_damage_share).values[0],
            (player_6.wards_placed).values[0],
            (player_6.wards_destroyed).values[0],           
            (player_7.participant_id).values[0],
            (player_7.champion_name).values[0],
            (player_7.role).values[0],
            (player_7.level).values[0],
            (player_7.kills).values[0],
            (player_7.deaths).values[0],
            (player_7.assists).values[0],
            (player_7.total_gold_earned).values[0],
            (player_7.creep_score).values[0],
            (player_7.kill_participation).values[0],
            (player_7.champion_damage_share).values[0],
            (player_7.wards_placed).values[0],
            (player_7.wards_destroyed).values[0],           
            (player_8.participant_id).values[0],
            (player_8.champion_name).values[0],
            (player_8.role).values[0],
            (player_8.level).values[0],
            (player_8.kills).values[0],
            (player_8.deaths).values[0],
            (player_8.assists).values[0],
            (player_8.total_gold_earned).values[0],
            (player_8.creep_score).values[0],
            (player_8.kill_participation).values[0],
            (player_8.champion_damage_share).values[0],
            (player_8.wards_placed).values[0],
            (player_8.wards_destroyed).values[0],
            (player_9.participant_id).values[0],
            (player_9.champion_name).values[0],
            (player_9.role).values[0],
            (player_9.level).values[0],
            (player_9.kills).values[0],
            (player_9.deaths).values[0],
            (player_9.assists).values[0],
            (player_9.total_gold_earned).values[0],
            (player_9.creep_score).values[0],
            (player_9.kill_participation).values[0],
            (player_9.champion_damage_share).values[0],
            (player_9.wards_placed).values[0],
            (player_9.wards_destroyed).values[0],
            (player_10.participant_id).values[0],
            (player_10.champion_name).values[0],
            (player_10.role).values[0],
            (player_10.level).values[0],
            (player_10.kills).values[0],
            (player_10.deaths).values[0],
            (player_10.assists).values[0],
            (player_10.total_gold_earned).values[0],
            (player_10.creep_score).values[0],
            (player_10.kill_participation).values[0],
            (player_10.champion_damage_share).values[0],
            (player_10.wards_placed).values[0],
            (player_10.wards_destroyed).values[0],
            game_winner]
   
    df_5_minutes = df_5_minutes.append(pd.Series(data, index=name_of_columns), ignore_index=True)


In [174]:
df_5_minutes = pd.get_dummies(df_5_minutes, columns=["patch_version"], prefix="patch_version_")

p1 = df_5_minutes['champion_name_player_1'].unique()
p2 = df_5_minutes['champion_name_player_2'].unique()
p3 = df_5_minutes['champion_name_player_3'].unique()
p4 = df_5_minutes['champion_name_player_4'].unique()
p5 = df_5_minutes['champion_name_player_5'].unique()
p6 = df_5_minutes['champion_name_player_6'].unique()
p7 = df_5_minutes['champion_name_player_7'].unique()
p8 = df_5_minutes['champion_name_player_8'].unique()
p9 = df_5_minutes['champion_name_player_9'].unique()
p10 =df_5_minutes['champion_name_player_10'].unique()

hero_names = np.unique(np.concatenate([p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]))

new_vals = [0 for i in range(len(hero_names))]
df_5_minutes = df_5_minutes.reindex(columns=df_5_minutes.columns.tolist() + hero_names.tolist())   # add empty cols
df_5_minutes[hero_names] = new_vals  # multi-column assignment works for existing cols

def fill_champion(df_5_minutes):
    df_5_minutes[df_5_minutes.champion_name_player_1] = -1
    df_5_minutes[df_5_minutes.champion_name_player_2] = -1
    df_5_minutes[df_5_minutes.champion_name_player_3] = -1
    df_5_minutes[df_5_minutes.champion_name_player_4] = -1
    df_5_minutes[df_5_minutes.champion_name_player_5] = -1

    df_5_minutes[df_5_minutes.champion_name_player_6] = 1
    df_5_minutes[df_5_minutes.champion_name_player_7] = 1
    df_5_minutes[df_5_minutes.champion_name_player_8] = 1
    df_5_minutes[df_5_minutes.champion_name_player_9] = 1
    df_5_minutes[df_5_minutes.champion_name_player_10] = 1

    return df_5_minutes

df_5_minutes = df_5_minutes.apply(fill_champion, axis=1)

print(df_5_minutes.shape)
df_5_minutes.head()

(665, 314)


,game_external_ref,blue_team_ref,red_team_ref,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,red_team_total_gold,red_team_inhibitors,red_team_towers,red_team_barons,red_team_total_kills,game_timestamp,game_number,team_a_side,team_b_side,team_a_record_wins,team_b_record_wins,team_a_record_losses,team_b_record_losses,best_of,red_team_dragons,blue_team_dragons,participant_id_player_1,champion_name_player_1,role_player_1,level_player_1,kills_player_1,deaths_player_1,assists_player_1,total_gold_earned_player_1,creep_score_player_1,kill_participation_player_1,champion_damage_share_player_1,wards_placed_player_1,wards_destroyed_player_1,participant_id_player_2,champion_name_player_2,role_player_2,...,Shen,Shyvana,Sion,Sivir,Skarner,Soraka,Swain,Sylas,Syndra,TahmKench,Taliyah,Taric,Thresh,Tristana,Trundle,Tryndamere,TwistedFate,Twitch,Udyr,Urgot,Varus,Vayne,Veigar,Velkoz,Viego,Viktor,Vladimir,Volibear,Xayah,Xerath,XinZhao,Yasuo,Yone,Yorick,Yuumi,Zac,Zed,Ziggs,Zilean,Zoe
0,104727755403883501,103877675241047720,103935523328473675,0,0,0,0,0,0,0,0,0,0,2020-08-27 16:52:00.000000,1,blue,red,4,4,2,2,1,0,0,1,Shen,top,1,0,0,0,0,0,0.0,0.0,0,0,2,Hecarim,jungle,...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,104727755403949039,104710359142494900,103877554248683116,0,0,0,0,0,0,0,0,0,0,2020-08-27 15:57:00.000000,1,blue,red,2,2,4,4,1,0,0,1,Sett,top,1,0,0,0,0,0,0.0,0.0,0,0,2,Lillia,jungle,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,104727755403949049,103935523328473675,103877675241047720,0,0,0,0,0,0,0,0,0,0,2020-09-10 20:35:00.000000,1,blue,red,4,4,2,2,1,0,0,1,Ornn,top,1,0,0,0,0,0,0.0,0.0,0,0,2,Kindred,jungle,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,104727755403949055,103877675241047720,104710359142494900,0,0,0,0,0,0,0,0,0,0,2020-09-10 17:44:00.000000,1,blue,red,4,2,2,4,1,0,0,1,Renekton,top,1,0,0,0,0,0,0.0,0.0,0,0,2,Nidalee,jungle,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,104727755403949063,103877625775457850,104710369093939895,0,0,0,0,0,0,0,0,0,0,2020-08-27 19:40:00.000000,1,blue,red,4,0,3,6,1,0,0,1,Urgot,top,1,0,0,0,0,0,0.0,0.0,0,0,2,Sett,jungle,...,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


,game_external_ref,blue_team_ref,red_team_ref,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,red_team_total_gold,red_team_inhibitors,red_team_towers,red_team_barons,red_team_total_kills,game_timestamp,game_number,team_a_side,team_b_side,team_a_record_wins,team_b_record_wins,team_a_record_losses,team_b_record_losses,best_of,red_team_dragons,blue_team_dragons,participant_id_player_1,champion_name_player_1,role_player_1,level_player_1,kills_player_1,deaths_player_1,assists_player_1,total_gold_earned_player_1,creep_score_player_1,kill_participation_player_1,champion_damage_share_player_1,wards_placed_player_1,wards_destroyed_player_1,participant_id_player_2,champion_name_player_2,role_player_2,...,Shen,Shyvana,Sion,Sivir,Skarner,Soraka,Swain,Sylas,Syndra,TahmKench,Taliyah,Taric,Thresh,Tristana,Trundle,Tryndamere,TwistedFate,Twitch,Udyr,Urgot,Varus,Vayne,Veigar,Velkoz,Viego,Viktor,Vladimir,Volibear,Xayah,Xerath,XinZhao,Yasuo,Yone,Yorick,Yuumi,Zac,Zed,Ziggs,Zilean,Zoe


In [182]:
del df_5_minutes['champion_name_player_1']
del df_5_minutes['champion_name_player_2']
del df_5_minutes['champion_name_player_3']
del df_5_minutes['champion_name_player_4']
del df_5_minutes['champion_name_player_5']
del df_5_minutes['champion_name_player_6']
del df_5_minutes['champion_name_player_7']
del df_5_minutes['champion_name_player_8']
del df_5_minutes['champion_name_player_9']
del df_5_minutes['champion_name_player_10']
del df_5_minutes['role_player_1']
del df_5_minutes['role_player_2']
del df_5_minutes['role_player_3']
del df_5_minutes['role_player_4']
del df_5_minutes['role_player_5']
del df_5_minutes['role_player_6']
del df_5_minutes['role_player_7']
del df_5_minutes['role_player_8']
del df_5_minutes['role_player_9']
del df_5_minutes['role_player_10']
del df_5_minutes['blue_team_ref']
del df_5_minutes['red_team_ref']
del df_5_minutes['game_external_ref']
del df_5_minutes['game_timestamp']
del df_5_minutes['team_a_side']
del df_5_minutes['team_b_side']

In [176]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d
from sklearn import preprocessing

In [183]:
df_5_minutes = move_columns(df_5_minutes, ['game_winner'], 0)
X = df_5_minutes.iloc[:,1:]
y = df_5_minutes.iloc[:,:1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=33)

y_train = column_or_1d(y_train, warn=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


In [184]:
scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_scaled = scaler.transform(X_train)

clf = LogisticRegression(max_iter=500).fit(X_scaled, y_train)
X_test_scaled = scaler.transform(X_test)
print(clf.score(X_test_scaled, y_test))

ValueError: ignored

In [169]:
clf = RandomForestClassifier(n_estimators=115, max_features=200, bootstrap= False)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.745


In [161]:
clf = KNeighborsClassifier(n_neighbors=30)
clf.fit(X_scaled, y_train)
print(clf.score(X_test_scaled, y_test))

0.75


In [162]:
clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=None)
clf.fit(X_scaled, y_train)
print(clf.score(X_test_scaled, y_test))

0.755
